# Loading Data

In [1]:
import tensorflow as tf
# Loading MNIST Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
working_dir = '/Users/ndjido/Documents/Davidson_Consulting/TF_DL_Meetup/Demo/ConvNet_MNIST/'

### Summary function

In [3]:
def variable_summaries(var, name):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean/' + name, mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev/' + name , stddev)
        tf.summary.scalar('max/' + name, tf.reduce_max(var))
        tf.summary.scalar('min/' + name, tf.reduce_min(var))
        tf.summary.histogram('histogram/' + name, var)

###  Network Building-block Helpers

In [4]:
def create_conv_layer(inputs, inputs_channels_dim, patch_dim, output_channels_dim, stride_x=1, stride_y=1, name="ConvLayer", activation=False):
    with tf.name_scope(name):
        W = tf.Variable(tf.truncated_normal([patch_dim, patch_dim, inputs_channels_dim, output_channels_dim], stddev=0.1), name="W")
        variable_summaries(W, 'Weights')
        b = tf.Variable(tf.truncated_normal([output_channels_dim]), name="b")
        variable_summaries(b, 'Biases')
        
        layer = tf.nn.conv2d(inputs, W, strides=[1, stride_x, stride_y, 1], padding="SAME") + b
        if activation == True:
            return tf.nn.relu(layer)
        else:
            return layer
        
def create_polling_layer(conv_layer, _padding="SAME", stride_x=2, stride_y=2, name="PollingLayer"):
    with tf.name_scope(name):
        return tf.nn.relu(tf.nn.max_pool(conv_layer, ksize=[1, stride_x, stride_y, 1], strides=[1, stride_x, stride_y, 1], padding=_padding))

def create_fully_connected_layer(inputs, inputs_dim, output_dim, name="fully_connected_layer", p_dropout=0):
     with tf.name_scope(name):
        W = tf.Variable(tf.truncated_normal([inputs_dim, output_dim], stddev=0.1), name="W")
        variable_summaries(W, 'Weights')
        b = tf.Variable(tf.truncated_normal([output_dim]), name="b")
        variable_summaries(b, 'Biases')
        layer = tf.nn.softmax(tf.matmul(inputs, W) + b)
        if p_dropout > 0.0 and p_dropout < 1.0:
            return tf.nn.dropout(layer, p_dropout)
        else:
            return layer

def create_softmax_layer(inputs, inputs_dim, output_dim, name="softmax_layer"):
     with tf.name_scope(name):
        W = tf.Variable(tf.truncated_normal([inputs_dim, output_dim], stddev=0.1), name="W")
        variable_summaries(W, 'Weights')
        b = tf.Variable(tf.truncated_normal([output_dim]), name="b")
        variable_summaries(b, 'Biases')
        return tf.nn.relu(tf.matmul(inputs, W) + b)
        

## Building the Network

In [5]:
# Input Dimensions
A = 6
B = 12
C = 24
D = 200

nb_pixels_x = 28
nb_pixels_y = nb_pixels_x
_inputs_channels_dim = 1
_output_dim = 10

X = tf.placeholder(tf.float32, shape=[None, nb_pixels_x * nb_pixels_y])
X_image = tf.reshape(X, [-1, nb_pixels_x, nb_pixels_y, _inputs_channels_dim])
Y_ = tf.placeholder(tf.float32, shape=[None, _output_dim])   

Y1 = create_conv_layer(X_image, _inputs_channels_dim, 6, A, 1, 1, name="ConvLayer_A", activation=True)
Y2 = create_conv_layer(Y1, A, 5, B, 2, 2, name="ConvLayer_B", activation=True)
Y3 = create_conv_layer(Y2, B, 4, C, 2, 2, name="ConvLayer_C", activation=True)

Y3_ = tf.reshape(Y3, shape=[-1, 7 * 7 * C])
Y4 = create_fully_connected_layer(Y3_, 7 * 7 * C, D, name="fully_connected_layer")

Y = create_softmax_layer(Y4, D, _output_dim, name="softmax_layer")

## Loss Function Operation

In [6]:
with tf.name_scope('cross_entropy'):
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=Y_))

tf.summary.scalar("cross_entropy_loss", cross_entropy_loss)

<tf.Tensor 'cross_entropy_loss:0' shape=() dtype=string>

## Training Operation

In [7]:
with tf.name_scope("Trainer"):
    trainer = tf.train.AdamOptimizer(.03).minimize(cross_entropy_loss)

## Accuracy Operation

In [8]:
with tf.name_scope('Accuracy'):
    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

## Runing the training of the model

In [ ]:
with tf.Session() as sess:
    #Training
        
    sess.run(tf.global_variables_initializer())
    
    # Summary writer for TensorBoad  
    merged_summary = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(working_dir + 'train', sess.graph)
    test_writer = tf.summary.FileWriter(working_dir + 'test')
    
    for i in range(10000):
        # Train
        x_data, y_data = mnist.train.next_batch(100)
        summary, _ = sess.run([merged_summary, trainer], feed_dict={X: x_data, Y_: y_data})
        train_writer.add_summary(summary, i)
        # Test
        summary, acc = sess.run([merged_summary, accuracy], feed_dict={X: mnist.test.images, Y_: mnist.test.labels})
        test_writer.add_summary(summary, i)
        print('Step %s -> Accuracy %s' % (i, acc))

## Accuracy = 98.5%